<a href="https://colab.research.google.com/github/s-pike3/Projects_In_AI-ML/blob/main/HW6_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sarah Pike \\
Projects in AI&ML \\
Homework 6, Part 1

In [1]:
import numpy as np
from torch.utils.data.dataloader import default_collate
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn import tree
import matplotlib.pyplot as plt
import math
import torch
from torch import nn
from torch.nn import functional as F
!pip install datasets
from datasets import load_dataset
from tokenizers import ByteLevelBPETokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers import normalizers, pre_tokenizers, Tokenizer, models, trainers
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

### Data Preparation

Load MultiNews Dataset from Hugging Face

In [3]:
from datasets import load_dataset
dataset = load_dataset("alexfabbri/multi_news",trust_remote_code=True)

train.src.cleaned:   0%|          | 0.00/548M [00:00<?, ?B/s]

train.tgt:   0%|          | 0.00/58.8M [00:00<?, ?B/s]

val.src.cleaned:   0%|          | 0.00/66.9M [00:00<?, ?B/s]

val.tgt:   0%|          | 0.00/7.30M [00:00<?, ?B/s]

test.src.cleaned:   0%|          | 0.00/69.0M [00:00<?, ?B/s]

test.tgt:   0%|          | 0.00/7.31M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Take smaller subset of training and test datasets for runtime purposes

In [24]:
train_split = dataset['train'].train_test_split(test_size=0.999)
trainset = train_split['train']

test_split = dataset['test'].train_test_split(test_size=0.99)
testset = test_split['train']

In [5]:
from transformers import AutoTokenizer, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
from accelerate.utils.memory import clear_device_cache

clear_device_cache()

Perform Tokenization

In [7]:
from transformers import AutoTokenizer

def tokenize_doc(examples):
    return tokenizer(examples["document"], max_length=1024,padding="max_length", truncation=True)

def tokenize_summary(examples):
    return tokenizer(examples["summary"], max_length=1024,padding="max_length", truncation=True)



In [8]:
tokenized_trainset = trainset.map(tokenize_doc, batched=True)
tokenized_trainset = tokenized_trainset.rename_column("input_ids", "labels")
tokenized_trainset = tokenized_trainset.map(tokenize_summary, batched=True)
tokenized_trainset = tokenized_trainset.remove_columns(["document","summary"])
tokenized_trainset.set_format("torch")

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

In [9]:
tokenized_testset = testset.map(tokenize_doc, batched=True)
tokenized_testset = tokenized_testset.rename_column("input_ids", "labels")
tokenized_testset = tokenized_testset.map(tokenize_summary, batched=True)
tokenized_testset = tokenized_testset.remove_columns(["document","summary"])
tokenized_testset.set_format("torch")

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Load Data

In [10]:
batch_size = 1
trainloader = torch.utils.data.DataLoader(tokenized_trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(tokenized_testset, batch_size=batch_size)

Set up optimizer and learning rate scheduler

In [11]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [12]:
from transformers import get_scheduler

num_epochs = 1
num_training_steps = num_epochs * len(trainloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [13]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [14]:
clear_device_cache()

Fine Tune Model

In [15]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in trainloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/44 [00:00<?, ?it/s]

#### Evaluation

#### Bleu

In [18]:
import evaluate

metric = evaluate.load("bleu")
model.eval()
for batch in testloader:
    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    summary = tokenizer.batch_decode(predictions, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    actual = tokenizer.batch_decode(list(batch.items())[0][1], skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    metric.add_batch(predictions=[summary], references=[[actual]])


metric.compute()

{'bleu': 0.15767609687624937,
 'precisions': [0.5966212211025489,
  0.22378213030178515,
  0.09545142857142858,
  0.05215819105598023],
 'brevity_penalty': 0.9819934643926581,
 'length_ratio': 0.9821536532389887,
 'translation_length': 43862,
 'reference_length': 44659}

#### Rouge

In [23]:
metric = evaluate.load('rouge')
model.eval()
for batch in testloader:
    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    summary = tokenizer.batch_decode(predictions, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    actual = tokenizer.batch_decode(list(batch.items())[0][1], skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    metric.add_batch(predictions=[summary], references=[[actual]])


metric.compute()

{'rouge1': np.float64(0.5854624287310547),
 'rouge2': np.float64(0.20186864242106456),
 'rougeL': np.float64(0.4030647732994317),
 'rougeLsum': np.float64(0.5530302573820696)}

#### Discussion

Testing indicates the rouge score is significantly higher than the bleu score. This can be justified because rouge is primarily used for evaluating text summarization and bleu is used for machine translation. Since this task involves text summarization, rouge is the more relevant metric.

Selection of different LLMS would likely yield different performance. Since I selected the BART model fine-tuned on the CNN News dataset, it is likely to have higher performance than a more general model.

Hyperparameters to consider are learning rate, batch size, and number of epochs. I chose the learning rate schedule recommended by hugging face's BART fine-tuning tutorial. Although using a batch size greater than 1 could have potentially improved model performance, this was not feasible due to memory constraints. Training for a greater number of epochs may have improved model performance but might also have led to overfitting.


### Works Cited

Bart. Hugging Face. (n.d.). https://huggingface.co/docs/transformers/en/model_doc/bart

Fine-tune a pretrained model. Hugging Face. (n.d.-b). https://huggingface.co/docs/transformers/en/training

Sharma, S. (2024, February 17). Text summarization with Bart Model. Medium. https://medium.com/@sandyeep70/demystifying-text-summarization-with-deep-learning-ce08d99eda97